# 라이브러리 불러오기

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree

In [2]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 데이터 불러오기

In [3]:
data_path: str = "~/house/data"
train_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "train.csv"))
test_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv"))
sample_submission: pd.DataFrame = pd.read_csv(os.path.join(data_path, "sample_submission.csv"))

In [4]:
# 금리, 지하철, 학교, 공원 정보 불러오기
interest_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "interestRate.csv"))
subway_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "subwayInfo.csv"))
school_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "schoolinfo.csv"))
park_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "parkInfo.csv"))

# 데이터 병합

## 금리 데이터 병합
* `interest_data`: 2018년 12월 ~ 2024년 5월까지의 금리
* 계약 연월 기준으로 `interest_data`를 `train_data`로 병합 (2019년 4월 ~ 2023년 12월)
* 계약 연월 기준으로 `interest_data`를 `test_data`로 병합 (2024년 1월 ~ 2024년 6월)

In [ ]:
# 계약 연월 기준으로 interest_data를 train_data로 병합
merged_train = pd.merge(train_data, interest_data, left_on="contract_year_month", right_on="year_month", how="left")
merged_train = merged_train.drop(columns=["year_month"])
merged_train

In [ ]:
merged_test = pd.merge(test_data, interest_data, left_on="contract_year_month", right_on="year_month", how="left")
merged_test = merged_test.drop(columns=["year_month"])
merged_test

In [ ]:
# 금리 결측치 개수 확인 (2024년 6월)
merged_test["interest_rate"].isnull().sum()

## 최단거리 데이터 병합

### sklearn의 BallTree를 활용한 haversine 거리 계산 함수

In [8]:
def find_closest_distance_haversine(
    train_data: pd.DataFrame, 
    loc_df: pd.DataFrame
) -> pd.DataFrame:
    """건물과 지하철/학교/공원 사이의 haversine 거리를 계산하는 함수

    Args:
        train_data (pd.DataFrame): 학습(훈련) 또는 테스트 데이터프레임
        loc_df (pd.DataFrame): 위도, 경도를 column으로 갖는 데이터프레임

    Returns:
        pd.DataFrame: index, 위도, 경도, haversine 거리를 column으로 갖는 반환
    """
    # degree->radian 값으로 변환 for 삼각함수
    train_coords = np.radians(train_data[["latitude", "longitude"]].values)
    loc_coords = np.radians(loc_df[["latitude", "longitude"]].values)
    
    # Ball Tree 생성 
    tree = BallTree(loc_coords, metric="haversine")

    distances, indices = tree.query(train_coords, k=1) # 가까운 1 지점만 
    distances_meter = distances * 6371000 # 단위를 meter로 변환

    closest_coords = loc_df[["latitude", "longitude"]].iloc[indices.flatten()].values # 가까운 지점 좌표

    # index, 최단거리, 최단거리에 해당하는 지점의 위도, 경도로 이루어진 데이터프레임 생성
    result_df = pd.DataFrame({
        "index" : train_data.index,
        "closest_distance" : distances_meter.flatten(),
        "closest_latitude" : closest_coords[:, 0],
        "closest_longtitude" : closest_coords[:, 1]
    })

    return result_df


### subway 병합

In [9]:
subway_result = find_closest_distance_haversine(train_data, subway_data)
subway_result.columns = ["index", "nearest_subway_distance", "nearest_subway_latitude", "nearest_subway_longtitude"]
train_data = pd.merge(train_data, subway_result, on="index")

In [10]:
subway_result = find_closest_distance_haversine(test_data, subway_data)
subway_result.columns = ["index", "nearest_subway_distance", "nearest_subway_latitude", "nearest_subway_longtitude"]
test_data = pd.merge(test_data, subway_result, on="index")

### school 병합

In [11]:
school_result = find_closest_distance_haversine(train_data, school_data)
school_result.columns = ["index", "nearest_school_distance", "nearest_school_latitude", "nearest_school_longtitude"]
train_data = pd.merge(train_data, school_result, on="index")

In [12]:
school_result = find_closest_distance_haversine(test_data, school_data)
school_result.columns = ["index", "nearest_school_distance", "nearest_school_latitude", "nearest_school_longtitude"]
test_data = pd.merge(test_data, school_result, on="index")

### park 병합

In [13]:
park_result = find_closest_distance_haversine(train_data, park_data)
park_result.columns = ["index", "nearest_park_distance", "nearest_park_latitude", "nearest_park_longtitude"]
train_data = pd.merge(train_data, park_result, on="index")

In [14]:
park_result = find_closest_distance_haversine(test_data, park_data)
park_result.columns = ["index", "nearest_park_distance", "nearest_park_latitude", "nearest_park_longtitude"]
test_data = pd.merge(test_data, park_result, on="index")

## 병합한 데이터 확인

In [ ]:
on = merged_train.columns.drop("interest_rate").tolist() # 병합 기준이 될 column 리스트
train_data = pd.merge(merged_train, train_data, on=on, how="left")
# train_data = train_data.drop(columns=["index"])
train_data

In [ ]:
on = merged_test.columns.drop("interest_rate").tolist() # 병합 기준이 될 column 리스트
test_data = pd.merge(merged_test, test_data, on=on, how="left")
# test_data = test_data.drop(columns=["index"])
test_data

In [17]:
# 이상치 탐지 함수
def find_outliers(data: pd.Series) -> pd.Series:
    """안 울타리(inner fence) 밖에 있는 데이터(이상치, outlier)를 반환하는 함수

    Args:
        data (pd.Series): 이상치 탐지를 하고싶은 데이터의 column

    Returns:
        pd.Series: 이상치에 해당하는 데이터 Series 반환
    """
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data < lower_bound) | (data > upper_bound)]

# EDA

In [18]:
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering

In [ ]:
eda_df = train_data.copy()
eda_df

### 1. 결측치 확인

결측치 확인 결과 없음을 확인

In [ ]:
# 각 열에서 누락된 값의 수를 계산
missing_values = eda_df.isnull().sum()

# 누락된 값의 백분율 계산
missing_percentage = (missing_values / len(eda_df)) * 100

# 누락된 값 비율을 기준으로 열 정렬
sorted_missing_percentage = missing_percentage.sort_values(ascending=False)
sorted_missing_percentage

In [ ]:
# 각 열에서 누락된 값의 수를 계산
missing_values = test_data.isnull().sum()

# 누락된 값의 백분율 계산
missing_percentage = (missing_values / len(test_data)) * 100

# 누락된 값 비율을 기준으로 열 정렬
sorted_missing_percentage = missing_percentage.sort_values(ascending=False)
sorted_missing_percentage

### 2. 상관관계 분석

deposit과 상관관계가 높은 feature 확인

: area_m2, nearest_subway_distance가 유의미한 상관관계(앞은 음, 뒤는 양의 상관관계)를 가진다고 판단

In [ ]:
eda_df.drop(columns="index", inplace=False).corr()["deposit"].abs().sort_values(ascending =False).head(20)

#### 2.1. area_m2 범주에 따른 deposit  분포

In [25]:
# 면적 범주화 함수
def categorize_area(x):
    range_start = (x // 50) * 50
    range_end = range_start + 49
    return f"{range_start} - {range_end}"

In [ ]:
eda_df["area_m2_category"] = eda_df["area_m2"].apply(categorize_area)
print("범주화 결과 :", eda_df["area_m2_category"].unique())

##### 2.1.1 Box Plot

In [ ]:
trade_counts = eda_df["area_m2_category"].value_counts().reset_index()
trade_counts.columns = ["area_m2_category", "transaction_count"]

plt.figure(figsize=(12, 6))
sns.barplot(data=trade_counts, x="area_m2_category", y="transaction_count")
plt.title("Number of Transactions by area_m2_category")
plt.xlabel("area_m2_category")
plt.ylabel("Number of Transactions")
plt.xticks(rotation=45)
plt.grid()
plt.show()

- 50 - 99 범주 거래량이 가장 많음

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=eda_df, x="area_m2_category", y="deposit")
plt.title("area_m2_category vs deposit")
plt.grid()
plt.show()

- 300 m² 범주에서 보증금이 1,000,000을 넘어가는 이상치 제거

In [29]:
# deposit_outliers = find_outliers(train_data["deposit"])
# train_data_cleaned = train_data[~train_data["deposit"].isin(deposit_outliers)]

# print("전체 데이터 개수:", train_data.shape[0])
# print("이상치 개수:", deposit_outliers.count())
# print("정상 데이터 개수:", train_data_cleaned.shape[0])

# 특정 값 이상 제거 (1,000,000 이상)
eda_df_cleaned = eda_df[eda_df["deposit"] < 1000000]

##### 2.1.2 Line Plot

In [ ]:
categories = ["0.0 - 49.0", "50.0 - 99.0", "100.0 - 149.0", "150.0 - 199.0", "200.0 - 249.0", "250.0 - 299.0", "300.0 - 349.0"]
eda_df_cleaned["area_m2_category"] = pd.Categorical(eda_df_cleaned["area_m2_category"], categories=categories, ordered=True)
mean_deposit = eda_df_cleaned.groupby("area_m2_category")["deposit"].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=mean_deposit, x="area_m2_category", y="deposit", marker="o")
plt.title("Average Deposit by area_m2_category")
plt.grid()
plt.show()

- 면적이 증가함에 따라 보증금이 상승하는 경향 (양의 상관관계)
- 구간별 상승률이 일정하지 않음

In [ ]:
# 절대적인 변화량 계산
mean_deposit["absolute_change"] = mean_deposit["deposit"] - mean_deposit["deposit"].shift(1)

# 시각화
plt.figure(figsize=(12, 6))
sns.barplot(data=mean_deposit, x="area_m2_category", y="absolute_change")
plt.title("Average Deposit Absolute Change by area_m2_category")
plt.ylabel("Absolute Change")
plt.grid()
plt.show()

- 일반적으로 변화량은 비슷
- 150-199 ~ 200-249 사이 변화량 눈에 띄게 상승 (60평 전후)
- 250-249 ~ 300-349 사이 변화량 눈에 띄게 감소 (90평 전후)

##### 2.1.3 Insight

일반적으로 area_m2와 deposit은 양의 상관관계를 가지지만,

구간별로 변화량에 차이가 있어 추가 변수로 활용해도 좋을 듯 함

#### 2.2 area_m2와 상관관계가 높은 변수 활용

In [ ]:
eda_df.drop(columns=["index", "area_m2_category"], inplace=False).corr()["area_m2"].abs().sort_values(ascending =False).head(20)

### 3. 시계열 분석

#### 3.1 계약 연도별 및 월별 deposit 변화

특정 이벤트 발생에 따른 변화가 있어 보이지만, 외부 데이터라 활용 불가능

In [34]:
eda_df["contract_year"] = eda_df["contract_year_month"].astype(str).str[:4].astype(int)
eda_df["contract_month"] = eda_df["contract_year_month"].astype(str).str[4:6].astype(int)

In [ ]:
# 계약 월별 평균 전세가 계산
monthly_avg = eda_df.groupby(["contract_year", "contract_month"])["deposit"].mean().reset_index()

# 계약 연도, 월, 일을 결합하여 새로운 datetime 컬럼 생성
monthly_avg["contract_date"] = pd.to_datetime(monthly_avg["contract_year"].astype(str) + "-" + monthly_avg["contract_month"].astype(str) + "-01")

# 시각화
plt.figure(figsize=(12, 6))
plt.plot(monthly_avg["contract_date"], monthly_avg["deposit"], marker="o")
plt.title("Monthly Average Deposit Over Time")
plt.xlabel("Year-Month")
plt.ylabel("Average Deposit")
plt.grid()
plt.axvline(pd.Timestamp("2020-01-01"), color="red", linestyle="--", label="COVID-19 Start")
plt.axvline(pd.Timestamp("2022-01-01"), color="red", linestyle="--", label="COVID-19 End")
plt.axvline(pd.Timestamp("2022-03-09"), color="green", linestyle="--", label="President Election") # 부동산 규제 완화
plt.legend()
plt.show()

#### 3.2.시즌별 deposit 변화

대학입학 시즌 등 시즌별 변화가 있을 지 보았지만, 뚜렷한 변화는 안 보임

In [ ]:
# 계절을 나타내는 함수 정의
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Fall"

# 시각화
plt.figure(figsize=(8, 6))
eda_df.groupby(eda_df["contract_month"].apply(get_season))["deposit"].mean().plot(kind="bar")
plt.title("Average Deposit by Season")
plt.xlabel("Season")
plt.ylabel("Average Deposit")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.show()

### 4. 기타

#### 4.1. 지역별 범주화

##### 4.1.1. 지역별 deposit 분포 확인 

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=eda_df, x="longitude", y="latitude", hue="deposit", palette="viridis", size="area_m2", sizes=(20, 200), alpha=0.2)

plt.title("Deposit by Location")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

##### 4.1.1. area_m2 대비 deposit으로 분포 확인 

In [20]:
# 면적 대비 전세가 계산
eda_df["price_per_area"] = eda_df["deposit"] / eda_df["area_m2"]

In [21]:
Q1 = eda_df["price_per_area"].quantile(0.25)
Q2 = eda_df["price_per_area"].quantile(0.5)
Q3 = eda_df["price_per_area"].quantile(0.75)

def categorize_price(price_per_area):
    if price_per_area <= Q1:
        return "cheap"
    elif price_per_area <= Q2:
        return "normal"
    elif price_per_area <= Q3:
        return "expensive"
    else:
        return "super"

eda_df["price_category"] = eda_df["price_per_area"].apply(categorize_price)

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=eda_df, x="longitude", y="latitude", hue="price_category", palette="viridis", size="price_per_area", sizes=(20, 200), alpha=0.2)

plt.title("Deposit by Location")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

서울 중심 지역으로 갈수록 면적 대비 전세가 높아지는 것 확인 가능

##### 4.1.2. 클러스터링

In [22]:
# One-Hot Encoding
price_dummies = pd.get_dummies(eda_df["price_category"], prefix="category")
eda_cluster = pd.concat([eda_df[["longitude", "latitude", "price_per_area"]], price_dummies], axis=1)

K-Means

In [ ]:
# Elbow Method
sse = []
k_values = range(1, 15)

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(eda_cluster)
    sse.append(kmeans.inertia_)

# 결과 시각화
plt.figure(figsize=(10, 6))
plt.plot(k_values, sse, marker="o")
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.title("Elbow Method")
plt.xticks(k_values)
plt.grid()
plt.show()

In [24]:
# K-means 클러스터링 적용
kmeans = KMeans(n_clusters=3)
eda_df["kmeans_cluster"] = kmeans.fit_predict(eda_cluster)

In [ ]:
# 박스 플롯 시각화
plt.figure(figsize=(8, 4))
sns.boxplot(data=eda_df, x="kmeans_cluster", y="deposit")
plt.title("deposit by K-means Clusters")
plt.xlabel("kmeans_cluster")
plt.ylabel("deposit")
plt.grid()
plt.show()

In [ ]:
# 산점도 결과 시각화
plt.figure(figsize=(10, 6))
sns.scatterplot(data=eda_df, x="longitude", y="latitude", hue="kmeans_cluster", palette="viridis", alpha=0.2, size="price_per_area", sizes=(20, 200))
plt.title("Location Clustering with Price per Area")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend(title="Cluster")
plt.show()

##### 4.1.3. DBSCAN

DBSCAN(Density-Based Spatial Clustering of Applications with Noise)은 밀도 기반 클러스터링 알고리즘으로, 데이터 포인트의 밀도를 기준으로 클러스터를 형성

In [27]:
dbscan = DBSCAN(eps=0.5, min_samples=5)  # eps와 min_samples는 조정 가능
eda_df["dbscan_cluster"] = dbscan.fit_predict(eda_cluster)

In [ ]:
# 박스 플롯 시각화
plt.figure(figsize=(12, 8))
sns.boxplot(data=eda_df, x="dbscan_cluster", y="deposit")
plt.title("deposit by DBSCAN Clusters")
plt.xlabel("dbscan_cluster")
plt.ylabel("deposit")
plt.grid()
plt.show()

In [ ]:
# 산점도 결과 시각화
plt.figure(figsize=(10, 6))
sns.scatterplot(data=eda_df, x="longitude", y="latitude", hue="dbscan_cluster", palette="viridis", alpha=0.2, size="price_per_area", sizes=(20, 200))
plt.title("Location Clustering with Price per Area")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend(title="Cluster")
plt.show()

##### 4.1.4. Agglomerative 클러스터링

Agglomerative Clustering은 계층적 군집화 방법으로, 각 데이터 포인트를 개별 클러스터로 시작하여 점차적으로 클러스터를 합침

In [30]:
# Agglomerative Clustering 적용
agg_clustering = AgglomerativeClustering(n_clusters=3)
eda_df["agg_cluster"] = agg_clustering.fit_predict(eda_cluster)

In [ ]:
# 박스 플롯 시각화
plt.figure(figsize=(12, 8))
sns.boxplot(data=eda_df, x="agg_cluster", y="deposit")
plt.title("deposit by DBSCAN Clusters")
plt.xlabel("agg_cluster")
plt.ylabel("deposit")
plt.grid()
plt.show()

In [ ]:
# 산점도 결과 시각화
plt.figure(figsize=(10, 6))
sns.scatterplot(data=eda_df, x="longitude", y="latitude", hue="agg_cluster", palette="viridis", alpha=0.2, size="price_per_area", sizes=(20, 200))
plt.title("Location Clustering with Price per Area")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend(title="Cluster")
plt.show()

#### 4.2. contract_type별 평균 deposit 

In [ ]:
contract_type_avg = eda_df.groupby("contract_type")["deposit"].mean()
print(contract_type_avg)

크게 두드러지는 차이 없는 것 확인

# Data Preprocessing

In [44]:
train_tmp = train_data.copy()
test_tmp = test_data.copy()
# train_data = train_tmp.copy()
# test_data = test_tmp.copy()

### 1. built_year > 2024 행 삭제

In [ ]:
print("before train :", train_data.shape)
train_data = train_data[train_data["built_year"] < 2024]
print("after train :", train_data.shape)

### 2. 음수 층수 데이터 제거

In [46]:
# print("before train :", train_data.shape)
# train_data = train_data[train_data["floor"] >= 0]
# print("after train :", train_data.shape)

# Feature Engineering

In [ ]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
import optuna
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_data.columns

### 1. 사용하지 않는 컬럼 삭제

In [48]:
cols = ["index", "contract_type", "age", "interest_rate", "nearest_subway_latitude", "nearest_subway_longtitude", "nearest_school_latitude", "nearest_school_longtitude", "nearest_park_latitude", "nearest_park_longtitude"]

train_data.drop(columns=cols, inplace=True)
test_data.drop(columns=cols, inplace=True)

### 2. Feature 추가

#### 2.1. area_m2_category 컬럼 추가 (EDA-2.1)

In [49]:
def categorize_area(x):
    range_start = (x // 50) * 50
    range_end = range_start + 49
    return f"{range_start} - {range_end}"

In [50]:
area_m2_category = pd.get_dummies(train_data["area_m2"].apply(categorize_area), prefix="category") # One-Hot Encoding
train_data = pd.concat([train_data, area_m2_category], axis=1)

In [51]:
area_m2_category = pd.get_dummies(test_data["area_m2"].apply(categorize_area), prefix="category") # One-Hot Encoding
test_data = pd.concat([test_data, area_m2_category], axis=1)

In [52]:
train_data = train_data[train_data["category_300.0 - 349.0"] != True]
train_data.drop(columns=["category_300.0 - 349.0"], inplace=True)

#### 2.2. log 변환

In [53]:
train_data["log_deposit"] = np.log1p(train_data["deposit"])
train_data["log_area_m2"] = np.log1p(train_data["area_m2"])
train_data["log_school_distance"] = np.log1p(train_data["nearest_school_distance"])
train_data["log_park_distance"] = np.log1p(train_data["nearest_park_distance"])
train_data["log_subway_distance"] = np.log1p(train_data["nearest_subway_distance"])

train_data.drop(columns=["area_m2", "floor", "nearest_school_distance", "nearest_park_distance", "nearest_subway_distance"], inplace=True)

In [54]:
test_data["log_area_m2"] = np.log1p(test_data["area_m2"])
test_data["log_school_distance"] = np.log1p(test_data["nearest_school_distance"])
test_data["log_park_distance"] = np.log1p(test_data["nearest_park_distance"])
test_data["log_subway_distance"] = np.log1p(test_data["nearest_subway_distance"])

test_data.drop(columns=["area_m2", "floor", "nearest_school_distance", "nearest_park_distance", "nearest_subway_distance"], inplace=True)

### 3. 데이터 분리

In [55]:
X = train_data.drop(columns=["deposit", "log_deposit"], inplace=False)
y = train_data[["deposit", "log_deposit"]]

In [56]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
y_train.drop(columns=["deposit"], inplace=True)
y_valid.drop(columns=["log_deposit"], inplace=True)
print(f"y_train: {y_train.columns}")
print(f"y_valid: {y_valid.columns}")

# Modeling

In [58]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [59]:
# def objective(trial):
#     params = {
#         "n_estimators": trial.suggest_int("n_estimators", 50, 300),
#         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
#         "max_depth": trial.suggest_int("max_depth", 3, 10),
#         "subsample": trial.suggest_float("subsample", 0.5, 1.0),
#         "tree_method": "hist",
#         "device": "cuda",
#         "random_state": 42
#     }
    
#     model = xgb.XGBRegressor(**params)
    
#     score = cross_val_score(model, X_train, y_train, cv=kf, scoring="neg_mean_absolute_error")
#     return -score.mean()

# sampler = optuna.samplers.TPESampler(seed=42)
# xgb_study = optuna.create_study(direction="minimize", sampler=sampler)
# xgb_study.optimize(objective, n_trials=50)
# print("Best parameters for XGBoost: ", xgb_study.best_params)

In [60]:
best_params = {"n_estimators": 288, "learning_rate": 0.11112043349923437, "max_depth": 10, "subsample": 0.7511206505586165}

In [ ]:
# 최적화된 하이퍼파라미터를 반영한 모델 정의
best_model = xgb.XGBRegressor(**best_params, tree_method="gpu_hist", gpu_id=0, random_state=42)

scores = cross_val_score(best_model, X_train, y_train, cv=kf, scoring="neg_mean_absolute_error")
print(f"Voting Regressor: Mean MAE = {-scores.mean():.4f}, Std = {scores.std():.4f}")

In [ ]:
best_model.fit(X_train, y_train)
y_pred_log = best_model.predict(X_valid)
y_pred = np.expm1(y_pred_log) # 지수변환 (로그변환의 역변환)
mae = mean_absolute_error(y_valid, y_pred)
print(f"Final Model MAE: {mae:.4f}")

# Inference

In [64]:
X_test = test_data.copy()

In [66]:
y_test_log = best_model.predict(X_test)
y_test = np.expm1(y_test_log) # 지수변환 (로그변환의 역변환)

In [ ]:
# 지수변환 전/후 예측값 히스토그램 시각화
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
sns.histplot(y_test_log, ax=axes[0])
sns.histplot(y_test, ax=axes[1])
plt.show()

In [68]:
sample_submission["deposit"] = y_test
sample_submission.to_csv("output.csv", index=False)